In [211]:
import pandas as pd
import numpy as np
import sklearn.cluster
import distance
import csv
import re
import string
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import nltk
from collections import Counter
import pprint

In [212]:
pp = pprint.PrettyPrinter(indent=4)

In [213]:
with open('AnalysisData.csv') as data_file:    
    data = csv.DictReader(data_file, delimiter=',', quotechar='"')
    data.next
raw_data = open('AnalysisData.csv', 'rb')
reader = csv.reader(raw_data, delimiter=',', quoting=csv.QUOTE_MINIMAL)
x = list(reader)
for i in xrange(len(x)):
    del x[i][4]
    del x[i][3]
    del x[i][0]

http://chrisalbon.com/python/cleaning_text.html

In [214]:
for i in xrange(len(x)):
    from_word = x[i][0]
    
    if '-' in from_word and from_word.index('-') == 2:
        x[i][0] = from_word[3:].lower()

In [215]:
regex = re.compile('[%s]' % re.escape(string.punctuation))
for i in xrange(len(x)):
    
    tokenized_sms = word_tokenize(x[i][1])
    tokenized_sms_optimized = []
    
    for tokenized_word in tokenized_sms:

        new_tokenized_word = []
        for tokenized_character in tokenized_word: 
            new_tokenized_character = regex.sub('', tokenized_character)
            if not new_tokenized_character == '':
                new_tokenized_word.append(new_tokenized_character)
        if new_tokenized_word:
            new_tokenized_word = "".join(new_tokenized_word).lower()
            if not new_tokenized_word in stopwords.words('english'):
                tokenized_sms_optimized.append(new_tokenized_word)
    
    x[i][1] = " ".join(tokenized_sms_optimized)

In [216]:
data = np.array(x)
df = pd.DataFrame(x)

http://chrisalbon.com/python/pandas_list_unique_values_in_column.html

In [219]:
pp.pprint(df.shape)
from_unique_list = list(df.get(0).unique())
print("Unique From: " + str(len(from_unique_list)))
print("Unique SMS: " + str(len(df.get(1).unique())))

(6784, 2)
Unique From: 835
Unique SMS: 5601


In [247]:
from_unique_dict = {i:x for i, x in enumerate(from_unique_list)}
from_unique_dict.update({x:i for i, x in enumerate(from_unique_list)})

if len(from_unique_dict) == len(from_unique_list)*2:
    print("Successfully created unique dictionary from FROM data of sms")
else:
    print("Some FROM data is equal to number assigned to it")

from_count = Counter(df.get(0))
print("\nCount of sms from each sender")
pp.pprint(from_count.most_common(n=30))

sms_words_count = Counter()
df.get(1).str.lower().str.split().apply(sms_words_count.update)
print("\nCount of words in all sms")
pp.pprint(sms_words_count.most_common(n=30))

Successfully created unique dictionary from FROM data of sms

Count of sms from each sender
[   ('hdfcbk', 677),
    ('ipaytm', 396),
    ('kotakb', 329),
    ('shopcl', 311),
    ('axisbk', 235),
    ('domino', 169),
    ('myntra', 166),
    ('fchrge', 142),
    ('144555', 136),
    ('mobikw', 117),
    ('uberin', 103),
    ('shopmt', 101),
    ('airccm', 99),
    ('121', 98),
    ('020001', 87),
    ('56060', 87),
    ('650003', 83),
    ('goibib', 73),
    ('fpanda', 69),
    ('paybak', 55),
    ('52422999', 54),
    ('brgkng', 54),
    ('jioroam', 53),
    ('ebayin', 51),
    ('aircel', 50),
    ('366000', 49),
    ('koovss', 47),
    ('amazon', 46),
    ('jiopay', 44),
    ('600009', 41)]

Count of words in all sms
[   ('http', 1502),
    ('rs', 1476),
    ('card', 1186),
    ('get', 1185),
    ('call', 1076),
    ('order', 1029),
    ('paytm', 837),
    ('otp', 773),
    ('dear', 759),
    ('ending', 690),
    ('c', 669),
    ('4308', 663),
    ('code', 651),
    ('credit', 645),

In [234]:
from sklearn.feature_extraction.text import HashingVectorizer
hv = HashingVectorizer()
hv.transform(df.get(1))

<6784x1048576 sparse matrix of type '<type 'numpy.float64'>'
	with 108928 stored elements in Compressed Sparse Row format>

In [246]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(min_df=1)
tf_transform = vectorizer.fit_transform(df.get(1))
tf_transform

<6784x15667 sparse matrix of type '<type 'numpy.float64'>'
	with 108928 stored elements in Compressed Sparse Row format>